ASSIGNMENT 1: CAMERA CALIBRATION

Nel seguente notebook è stato implementato il codice necessario per la calibrazione di una camera tramite Direct Linear Transformation, ottenendo la matrice dei paramentri intrinseci(dipendenti da tipo e modello della camera, come lunghezza focale e aspect ratio dell'immagine), i parametri estrinseci(matrice di rotazione e vettore di traslazione che identificano la posa della camera nell'ambiente) e la matrice della camera P, necessaria per proiettare i punti tridimensionali del mondo in punti 2D nel piano immagine.
La matrice P verrà usata per proiettare sull'immagine del laboratorio tutti i punti appartenenti al pavimento(aventi coordinata Z=0); inoltre, al fine di misurare e quantificare la stima dei parametri ottenuti, verranno calcolati lo scarto quadratico medio e la deviazione standard della proiezione dei punti tramite la K-fold cross validation confrontando gli output dell'algoritmo implementato con quelli del metodo calibrateCamera contenuto nella libreria OpenCV2

Per l'implementazione del codice sono state utilizzate le slide del corso e le seguenti documentazioni:

numpy --> https://numpy.org/doc/1.24/reference/index.html#reference

scipy --> https://docs.scipy.org/doc/scipy/reference/index.html#scipy-api

opencv --> https://docs.opencv.org/4.x/index.html

Definiamo inizialmente la corrispondenza di ogni punto tra le sue coordinate 3D e quelle 2D.
Le coordinate 3D vengono prese rispetto al sistema di riferimento dell'immagine di calibrazione(il pattern a scacchiera nel nostro caso), mentre le coordinate 2D saranno i pixel corrispondenti.
Nello specifico la matrice objp conterrà le coordinate nello spazio(ogni riga identifica un punto con 3 coordinate), la matrice imgp conterrà le posizioni dei pixel(ogni riga identifica un pixel) 

In [2]:
import numpy as np
import scipy as sp
import cv2

'''
def define_Points()

Output
------
objp: Matrice 24X3 in cui ogni riga identifica un punto tramite le sue coordinate tridimensionali nel mondo
imgp: Matrice 24X2 in cui ogni riga identifica un punto tramite le coordinate del pixel corrispondente nel piano immagine

Tra objp e igmp c'è una corrispondenza 1 a 1, dove l'n-esima riga di objp e igmp identificano lo stesso punto in due sistemi di riferimento diversi
------
'''
def define_Points():
    nPunti = 24

    #Punti 3D
    objp = np.zeros((nPunti,3), np.float32)

    objp[1,:] =[7, 0, 0]
    objp[2,:] =[7, 0, 9]
    objp[3,:] =[0, 4, 5]
    objp[4,:] =[0, 3, 8]
    objp[5,:] =[4, 4, 9]
    
    #first face
    objp[1,:] =[7, 0, 0]
    objp[2,:] =[7, 0, 9]
    objp[3,:] =[0, 0, 7]
    objp[4,:] =[3, 0, 4]

    #second face
    objp[5,:] =[0, 5, 0]
    objp[6,:] =[0, 5, 7]
    objp[7,:] =[0, 0, 9]

    #top face
    objp[8,:] =[7, 5, 9]
    objp[9,:] =[4, 4, 9]

    #additional points
    objp[10,:] =[7, 0, 3]
    objp[11,:] =[0, 3, 7]
    objp[12,:] =[0, 5, 3]
    objp[13,:] =[7, 2, 9]
    
    objp[14,:] =[0, 5, 9]
    objp[15,:] =[1, 1, 9]
    objp[16,:] =[4, 3, 9]
    objp[17,:] =[5, 0, 5]
    objp[18,:] =[1, 0, 6]
    objp[19,:] =[4, 0, 8]
    objp[20,:] =[0, 2, 6]
    objp[21,:] =[0, 4, 1]
    objp[22,:] =[0, 4, 5]
    objp[23,:] =[0, 3, 8]

    #Punti 2D
    imgp = np.zeros((nPunti,2), np.float32)
    
    imgp[0,:] = [753, 599]
    imgp[1,:] = [870, 533]
    imgp[2,:] = [950, 329]
    imgp[3,:] = [702, 419]
    imgp[4,:] = [748, 353]
    imgp[5,:] = [811, 287]
    
    #first face
    imgp[0,:] = [753, 599]
    imgp[1,:] = [870, 533]
    imgp[2,:] = [950, 329]
    imgp[3,:] = [815, 429]
    imgp[4,:] = [842, 481]

    #second face
    imgp[5,:] = [654, 519]
    imgp[6,:] = [695, 352]
    imgp[7,:] = [833, 369]

    #top face
    imgp[8,:] = [838, 265]
    imgp[9,:] = [811, 287]

    #additional points
    imgp[10,:] = [895, 471]
    imgp[11,:] = [740, 381]
    imgp[12,:] = [669, 453]
    imgp[13,:] = [903, 299]
    
    imgp[14,:] = [710, 295]
    imgp[15,:] = [828, 344]
    imgp[16,:] = [834, 299]
    imgp[17,:] = [885, 441]
    imgp[18,:] = [825, 448]
    imgp[19,:] = [898, 374]
    imgp[20,:] = [755, 424]
    imgp[21,:] = [677, 514]
    imgp[22,:] = [702, 419]
    imgp[23,:] = [748, 353]
    
    return objp, imgp

Stampiamo i punti scelti sull'immagine

In [3]:
def show_points(imgp):
    # Let's show the selected points 
    img = cv2.imread("imgs/cubotest.jpeg")
    img_to_show = img.copy()
    for p in imgp:
        img_to_show = cv2.circle(img_to_show, p.astype(np.int32), 3, (255, 0, 0))

    cv2.imshow("img", img_to_show)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

img = cv2.imread("imgs/cubotest.jpeg")
objp, imgp = define_Points()
show_points(imgp)

Implementazione del metodo calibKRtp, che presi in input i punti 2D e 3D, restituisce le matrici dei parametri intrinseci ed estrinseci tramite la Direct Linear Transformation(DLT).

La calibrazione della camera tramite DLT viene effettuata con i seguenti passi:

1)Corrispondenza tra i punti 2D e 3D tramite matrice P:

Per ogni punto, la coppia di coordinate 2D-3D è legata dalla seguente equazione:

![image](eq1.png)

Possiamo scomporre questa equazione nel prodotto scalare tra un vettore di termini noti e un vettore di incognite, e ricomporre il sistema:

![image2](Ap.png)
    
2)Calcolo della matrice P:

Dal sistema precedente, possiamo trovare il vettore p imponendo Ap=0 vincolando ||p||=1. Ciò equivale a imporre che p sia nell'intorno di raggio 1 di una ipersfera centrata in 0, ottenendo $A^TAp - \lambda p^Tp=0$.

Questo problema può essere risolto trovando gli autovalori di $A^TA$ tramite la decomposizione SVD, in particolare l'autovettore corrispondente all'autovalore minore sarà l'ultima colonna di $V^T$.

Una volta trovato l'autovettore lo trasformo in una matrice 3x4:

![image3](p.png)

3)Applicazione della fattorizzazione RQ alla matrice P per trovare K=R e R=Q:

Posso vedere p come il seguente prodotto tra matrici:

![image4](rqfact.png)

In cui la matrice K è una triangolare superiore ed R è ortogonale; entrambe le matrici possono essere trovate tramite la fattorizzazione RQ(dove K=R ed R=Q)

4)Calcolo del vettore t:

Una volta ottenuta K, $t=K^{-1}*[p_{1,4},p_{2,4},p_{3,4}]^T$ dalla seguente uguaglianza:

![image5](t.png)

Inoltre ai fini dell'assignment non si è tenuto conto di nessun tipo di distorsione radiale o tangenziale


In [12]:
'''
def calibKRtp(imgp, objp)

Input
--------
imgp, objp: Matrici in cui ogni riga indica un punto rispettivamente con le sue coordinate nel piano imagine e nel mondo
--------

Output
--------
K: Matrice dei parametri intrinseci
R: Matrice di rotazione
t: Vettore di traslazione
p: Matrice camera
--------
'''

def calibKRtp(imgp, objp):
    
    A=np.zeros((2*imgp.shape[0], 12))

    #Creazione della matrice A, vista come il prodotto scalare tra il vettore dei termini noti(punti 2d e 3d) e il vettore delle incognite.
    #Ogni punto è rappresentato da 2 equazioni
    for i in range(imgp.shape[0]):
        xp, yp=imgp[i,:]
        X, Y, Z = objp[i,:]
        A[2*i,:]=[0,0,0,0,X,Y,Z,1,-yp*X,-yp*Y,-yp*Z,-yp]
        A[(2*i)+1,:]=[X,Y,Z,1,0,0,0,0,-xp*X,-xp*Y,-xp*Z,-xp]
        
    #Per risolvere il Constrained Least Square Problem si applica la SVD alla matrice A
    #Nello specifico l'autovettore associato all'autovalore minore sarà l'ultima colonna della trasposta della matrice v
    _, _, v = np.linalg.svd(A)
    praw=v[-1,:]/v[-1,-1]
    
    #Trasformazione del vettore p in matrice, si possono ottenere la sottomarice 3x3 e l'ultima colonna, necessari per calcolare i parametri intrinseci ed estrinseci,
    #tramite l'operazione di slicing sulla matrice p originaria
    idx=0
    p=np.reshape(praw,(3,4))
    #p=p/p[2,3]
    p13=p[:,:3]
    p4=p[:,-1]

    #Applicazione della fatorizzazione RQ alla sottomatrice 3x3 di p, con conseguente normalizzazione della matrice K in modo che K[3][3]=1
    #Il vettore t sarà ottenuto da K^1*p4.T
    K,R=sp.linalg.rq(p13)
    t=np.matmul(np.linalg.inv(K),p4.T)
    K=K/K[-1,-1]
    
    return K, R, t, p

K, R, t, p = calibKRtp(imgp, objp)
print("Matrice K(parametri intrinseci)")
print(K)
print()
print("Matrice R")
print(R)
print()
print("Vettore t")
print(t)

Matrice K(parametri intrinseci)
[[940.47288546   4.57944922 732.13661551]
 [  0.         937.93103191 484.88644406]
 [  0.           0.           1.        ]]

Matrice R
[[ 0.64986409 -0.72161352  0.23864324]
 [-0.29297704 -0.52755395 -0.79740284]
 [ 0.70131385  0.44828648 -0.55425456]]

Vettore t
[ 0.75849521  3.91750797 31.65745269]


Per valutare la bontà della stima della matrice K ottenuta tramite l'implementazione della calibrazione DLT, verranno mostrate le differenze assolute tra la matrice K ottenuta dalla nostra implementazione e la K ottenuta tramite il metodo cv2.calibrateCamera rispettivamente con 6, 12 e 24 punti di calibrazione sull'immagine di test.
Come parametro di distorsione non viene passato nulla.

Di seguito il calcolo della matrice K tramite il metodo di OpenCV e a seguire le differenze assolute:

In [13]:
#Fornisco una stima di K al metodo calibrateCamera
Kcv = np.zeros((3,3))
Kcv[0,0]=500
Kcv[1,1]=500
Kcv[2,2]=1
Kcv[0,2]=img.shape[0]/2
Kcv[1,2]=img.shape[1]/2

_, Kcv, _, _, _ = cv2.calibrateCamera([objp], [imgp], img.shape[0:2], Kcv, None, flags=cv2.CALIB_USE_INTRINSIC_GUESS|cv2.CALIB_FIX_S1_S2_S3_S4| cv2.CALIB_ZERO_TANGENT_DIST| cv2.CALIB_FIX_K2 | cv2.CALIB_FIX_K3 )

print(np.abs(K-Kcv))

[[ 83.9815763    4.57944922 365.21115456]
 [  0.          80.21135961 164.18434015]
 [  0.           0.           0.        ]]


![image](abs.png)


Vengono definite le funzioni projection e draw_cube_points, che sfruttano il metodo di OpenCV2 projectPoints per ottenere i corrispettivi punti nel piano immagine a partire dalle coordinate nel mondo e dai parametri intrinseci ed estrinseci

In [14]:
'''
def projection(objp, R, t, K)

Input
-------
objp: Matrice in cui ogni riga contine le coordinate nel mondo di un punto
R: Matrice di rotazione
t: Vettore di traslazione
K: Matrice dei parametri intrinseci
-------

Output
-------
imgpoints: Matrice che contiene le coordinate dei punti nel piano immagine
-------
'''

def projection(objp, R, t, K):
    imgpoints, _ = cv2.projectPoints(objp, R, t, K, np.zeros(5))
    imgpoints = np.squeeze(imgpoints)
    return imgpoints


'''
def draw_cube_points(K_, R_, t_)

Input
-------
K: Matrice dei parametri intrinseci
R: Matrice di rotazione
t: Vettore di traslazione
-------

'''
def draw_cube_points(K_, R_, t_):
    #Creazione di una griglia che rappresenta tutti i punti apartenenti al pattern a scacchiera
    points = []
    for x in range(8):
        for z in range(9):
            points.append([x, 0, z])

    for y in range(6):
        for z in range(10):
            points.append([0, y, z])
        
    for x in range(8):
        for y in range(6):
            points.append([x, y, 9])
               
    points = np.asarray(points).astype(np.float32)
    projected_points = projection(points, R_, t_, K_).astype(np.int32)
    img=cv2.imread("imgs/cubotest.jpeg")
    img_to_show_res = img.copy()
    
    #Stampa dei punti sull'immagine
    for p in projected_points:
        img_to_show_res = cv2.circle(img_to_show_res, p, 3, (255, 0, 0) )
    
    cv2.imshow("result", img_to_show_res)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

imgpoints = projection(objp, R, t, K)

#MSE tra i punti misurati e quelli ottenuti tramite il metodo projection
mean_error = (cv2.norm(imgp, imgpoints, normType=cv2.NORM_L2)**2)/len(imgpoints)
print( "total error: {}".format(mean_error) )

draw_cube_points(K, R, t)

total error: 4.285160889616237


Creazione dei due array che conterranno i punti presi nel sistema dell'immagine di riferimento e la loro corrispondenza in pixel

In [15]:
'''
def createWorldPoints()

Output
------
objp: Matrice 24X3 in cui ogni riga identifica un punto tramite le sue coordinate tridimensionali nel mondo
imgp: Matrice 24X2 in cui ogni riga identifica un punto tramite le coordinate del pixel corrispondente nel piano immagine
------
'''

def createWorldPoints():
    #n_corners = 13
    n_corners = 12
    
    #3D points
    objp = np.zeros((n_corners,3), np.float32)
    
    #objp contiene i punti presi nel sistema di riferimento del laboratorio
    #Punti a terra
    objp[1,:] = [0, 265, 0]
    objp[2,:] = [-94, 115, 0]
    objp[3,:] = [65, 60, 0]
    objp[4,:] = [0, 238, 0]
    objp[5,:] = [140, -72, 0]
    objp[6,:] = [-96, -132, 0]

    #Punti nello spazio
    objp[7, :] = [-135, 145, 70]
    objp[8, :] = [0, 320, 66]
    objp[9,:] = [65, 65, 80]
    objp[10,:] = [-43, 348, 128]
    objp[11,:] = [0, 348, 128]

    #2D points
    imgp = np.zeros((n_corners,2), np.float32)

    #imgp contiene le coordinate dei pixel dei punti
    imgp[0,:] = [604, 495]
    imgp[1,:] = [624, 306]
    imgp[2,:] = [453, 390]
    imgp[3,:] = [729, 438]
    imgp[4,:] = [622, 321]
    imgp[5,:] = [839, 584]
    imgp[6,:] = [345, 658]

    #Punti nello spazio
    imgp[7,:] = [381, 264]
    imgp[8,:] = [626, 202]
    imgp[9,:] = [737, 297]
    imgp[10,:] = [575, 116]
    imgp[11,:] = [630, 116]
    
    return objp, imgp

objS, imgS=createWorldPoints()

Calcolo della Camera Matrix p e stampa dei punti di calibrazione presi sull'immagine del laboratorio

In [16]:
KS, RS, tS, pS = calibKRtp(imgS, objS)
print("Camera matrix p:")
print(pS)
img2 = cv2.imread("imgs/stanzavuota.jpeg")
img_to_show = img2.copy()
for p in imgS:
    img_to_show = cv2.circle(img_to_show, p.astype(np.int32), 3, (255, 0, 0) )

cv2.imshow("img", img_to_show)
cv2.waitKey(0)
cv2.destroyAllWindows()


Camera matrix p:
[[ 2.16599000e+00  1.02745970e+00 -4.18595949e-01  6.00221385e+02]
 [ 2.64603063e-01 -2.26138041e-01 -1.94672540e+00  4.92219813e+02]
 [ 4.97033049e-04  1.51156643e-03 -7.42506612e-04  1.00000000e+00]]


Implementazione della K-fold cross validation:
Il dataset viene suddiviso in K parti chiamate fold, iterativamente il k-esimo fold verrà usato come set di validazione mentre i restanti saranno usati come training.
In questo modo ogni fold verrà utilizzato per la validazione del modello, riducendo quindi l'errore di overfitting dato dal riutilizzo degli stessi punti per la validazione

Funzione draw_ground che stampa dei punti appartenenti al pavimento, cioè tutti i punti con coordinata Z=0

In [17]:
'''
def draw_ground(img2, pS)

Input
------
img2: Immagine su cui verranno stampati i punti appartenenti al pavimento
pS: Matrice camera utilizzata per proiettare i punti 3D nel piano immagine
------
'''

def draw_ground(img2, pS):
    xg = np.arange(-200, 170, 8) 
    yg = np.arange(-300, 350, 8)
    #xg = np.arange(0, 10, 1) 
    #yg = np.arange(0, 10, 1)
    xx, yy = np.meshgrid(xg, yg)

    dim = xx.shape[0]*xx.shape[1]
    floor3d = np.zeros((dim,4), np.float32)

    xx = xx.reshape(dim)
    yy = yy.reshape(dim)

    floor3d[:,0] = xx 
    floor3d[:,1] = yy 
    floor3d[:,2] = np.zeros((dim))
    floor3d[:,3] = 1

    floorpx = np.zeros((floor3d.shape[0],3), np.float32)
    index=0
    for px in floorpx:
        px=pS@floor3d[index].T
        px=px/px[-1]
        floorpx[index]=px
        index+=1
    floorpx = np.squeeze(floorpx).astype(np.int32)
    ground=floorpx[:,:2]
    img_to_show_res = img2.copy()
    for p in ground:
        img_to_show_res = cv2.circle(img_to_show_res, p, 3, (255, 0, 0) )

    cv2.imshow("Punti pavimento(z=0)", img_to_show_res)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
draw_ground(img2, pS)

Definisco la funzione drawFold, che presa in input la matrice p calcolata sul training set e i punti del test set, restituisce il MSE tra i punti del test set e i loro corrispondenti trovati tramite la proiezione della matrice p

In [10]:
'''
def drawFold(p, testSet3d, testSetpx)

Input
------
p: Matrice camera precedentemente stimata tramite il training set, in questa funzione viene utilizzata per calcolare l'MSE tra i punti misurati in laboratorio e quelli ottenuti dalla proiezione
testSet3d: Testing set dei punti 3D utilizzati per ottenere la loro proiezione sul piano immagine tramite la matrice P
testSetpx: Testing set dei punti appartenenti al piano immagine utilizzati per calcolare la MSE con i punti ottenuti dalla matrice P
------

Output
------
mse: Mean Squared Error(Errore Quadratico Medio) tra i punti appartenenti al piano immagine misurati in laboratorio e quelli ottenuti dalla proiezione tramite matrice P
------
'''

def drawFold(p, testSet3d, testSetpx):
    
    index=0
    f11=np.zeros((3,4), np.float32)
    for x in testSet3d:
            f11[index]=np.append(testSet3d[index], 1)
            index+=1

    floorpx = np.zeros((testSet3d.shape[0],3), np.float32)
    index=0
    for px in floorpx:
        px=pS@f11[index].T
        px=px/px[-1]
        floorpx[index]=px
        index+=1
    floorpx = np.squeeze(floorpx).astype(np.int32)
    ground=floorpx[:,:2]
    imgfold = cv2.imread("imgs/stanzavuota.jpeg")
    img_to_show_res = imgfold.copy()
    for p in ground:
        img_to_show_res = cv2.circle(img_to_show_res, p, 3, (255, 0, 0) )

    testSetpx = np.squeeze(testSetpx).astype(np.int32)
    mse = (cv2.norm(testSetpx, ground, normType=cv2.NORM_L2)**2)/len(ground)

    cv2.imshow("Punti pavimento(K-fold)", img_to_show_res)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return mse
    

Creazione di una funzione di shuffle per ordinare casualmente i punti del dataset

In [11]:
'''
def shuffle(a, b)

Input
------
a, b: vettori che rappresentano il dataset dei punti
------

Output
------
shuffled_a, shuffled_b: vettori a e b con elementi ordinati casualmente
------
'''

def shuffle(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

puntiLab, pxLab = createWorldPoints()

puntiLab, pxLab=shuffle(puntiLab, pxLab)

Calcolo dell'MSE per ogni fold, MSE medio e deviazione standard

In [36]:
#Creazione dei 4 fold a partire dai punti misurati in laboratorio
f1 = puntiLab[0:3]
f2 = puntiLab[3:6]
f3 = puntiLab[6:9]
f4 = puntiLab[9:12]

p1 = pxLab[0:3]
p2 = pxLab[3:6]
p3 = pxLab[6:9]
p4 = pxLab[9:12]

#Stima di p e calcolo dell'MSE per ogni fold, con successivo calcolo di media e deviazione standard

#test = f1 
#training = f2 f3 f4
_, _, _, pfold= calibKRtp(pxLab[3:12], puntiLab[3:12])
mse1=drawFold(pfold, f1, p1)
print("MSE Fold 1: {0}".format(mse1))

#test = f2 
#training = f1 f3 f4
_, _, _, pfold= calibKRtp(np.concatenate((p1, p3, p4)), np.concatenate((f1, f3, f4)))
mse2=drawFold(pfold, f2, p2)
print("MSE Fold 2: {0}".format(mse2))

#test = f3
#training = f1 f2 f4
_, _, _, pfold= calibKRtp(np.concatenate((p1, p2, p4)), np.concatenate((f1, f2, f4)))
mse3=drawFold(pfold, f3, p3)
print("MSE Fold 3: {0}".format(mse3))

#test = f4 
#training = f1 f2 f3
_, _, _, pfold= calibKRtp(pxLab[0:8], puntiLab[0:8])
mse4=drawFold(pfold, f4, p4)
print("MSE Fold 4: {0}".format(mse4))

#Media MSE e deviazione standard
msetot=(mse1+mse2+mse3+mse4)/4
print("Mean MSE: {0}".format(msetot))
sigma=np.sqrt(((mse1-msetot)**2+(mse2-msetot)**2+(mse3-msetot)**2+(mse4-msetot)**2)/4)
print("Standard deviation: {0}".format(sigma))

MSE Fold 1: 310.3333333333333
MSE Fold 2: 294.6666666666667
MSE Fold 3: 11.0
MSE Fold 4: 13.333333333333336
Mean MSE: 157.33333333333334
Standard deviation: 145.27464411321827
